In [ ]:
import pandas as pd
import re
from collections import Counter

def extract_raw_opf(full):
    if not isinstance(full, str):
        return None
    
    m = re.match(r'^([^"«“„]+)', full.strip())
    if m:
        return m.group(1).strip()
    return None

raw_opfs = df["Повна назва"].apply(extract_raw_opf)
opf_counts = Counter(raw_opfs.dropna())

# беремо тільки популярні фрази
dynamic_opf_list = [opf for opf, count in opf_counts.items() if count > 10]
dynamic_opf_list = sorted(dynamic_opf_list, key=len, reverse=True)

print(dynamic_opf_list[:50])


import re

def split_opf_name(full):
    full = full.strip()
    
    # якщо весь текст у лапках — OPF нема
    if re.match(r'^[\"«“„].+[\"»”]$', full):
        return None, full.strip('«»“”"')
    
    for opf in dynamic_opf_list:
        if full.startswith(opf):
            name = full[len(opf):].strip().strip('«»“”"')
            return opf, name
    
    # fallback: не знайшли OPF
    return None, full.strip('«»“”"')

df["OPF"], df["FIRM_NAME"] = zip(*df["Повна назва"].apply(split_opf_name))


## Особи

In [ ]:
import ast
import pandas as pd

def parse_list_column(row_value):
    """Перетворює рядок зі списком dict у Python-список."""
    if not isinstance(row_value, str):
        return []
    try:
        return ast.literal_eval(row_value)
    except:
        return []


# -------------------------------
# 1) УПОВНОВАЖЕНІ ОСОБИ
# -------------------------------
def expand_authorized(df):
    max_items = 5
    out = {}

    for idx, row in df["Уповноважені особи"].items():
        people = parse_list_column(row)
        record = {}

        for i, p in enumerate(people[:max_items], start=1):
            record[f"Authorized_{i}_Name"] = p.get("ПІБ")
            record[f"Authorized_{i}_Role"] = p.get("Роль")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


# -------------------------------
# 2) ЗАСНОВНИКИ
# -------------------------------
def expand_founders(df):
    max_items = 5
    out = {}

    for idx, row in df["Засновники"].items():
        founders = parse_list_column(row)
        record = {}

        for i, f in enumerate(founders[:max_items], start=1):
            record[f"Founder_{i}_Name"] = f.get("ПІБ / Назва")
            record[f"Founder_{i}_Country"] = f.get("Країна")
            record[f"Founder_{i}_Contribution"] = f.get("Розмір внеску")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


# -------------------------------
# 3) БЕНЕФІЦІАРИ
# -------------------------------
def expand_beneficiaries(df):
    max_items = 5
    out = {}

    for idx, row in df["Бенефіціари"].items():
        bens = parse_list_column(row)
        record = {}

        for i, b in enumerate(bens[:max_items], start=1):
            record[f"Benef_{i}_Name"] = b.get("ПІБ")
            record[f"Benef_{i}_Country"] = b.get("Країна")
            record[f"Benef_{i}_Share"] = b.get("Частка")

        out[idx] = record

    return pd.DataFrame.from_dict(out, orient="index")


In [ ]:
df_auth = expand_authorized(df)
df_found = expand_founders(df)
df_benef = expand_beneficiaries(df)

df_final = pd.concat([df, df_auth, df_found, df_benef], axis=1)

df_final = df_final.drop(columns=[
    "Уповноважені особи",
    "Засновники",
    "Бенефіціари"
], errors="ignore")


# -

In [ ]:
# ------------------------
# Бенефіціари
# ------------------------

def expand_beneficiary(df: pd.DataFrame, )